## AUTOENCODER FOR DETECTION

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Define the dataset class
class VibrationDataset(Dataset):
    def __init__(self, csv_file):
        data = pd.read_csv(csv_file)
        self.scaler = MinMaxScaler()
        self.data = self.scaler.fit_transform(data[['x', 'y', 'z']].values)
        self.time_steps = data['time'].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32)

# Define the autoencoder model with added complexity
class ComplexAutoencoder(nn.Module):
    def __init__(self):
        super(ComplexAutoencoder, self).__init__()
        # Encoder with more layers and dropout
        self.encoder = nn.Sequential(
            nn.Linear(3, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        # Decoder with more layers and dropout
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Load datasets
non_fault_dataset = VibrationDataset('non_fault_data.csv')
fault_dataset = VibrationDataset('slow_expanding_fault_data.csv')

# Create data loaders
non_fault_loader = DataLoader(non_fault_dataset, batch_size=32, shuffle=True)

# Initialize the model, loss function, and optimizer
model = ComplexAutoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Train the autoencoder on non-fault data
num_epochs = 20
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for data in non_fault_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(non_fault_loader):.4f}")

# Evaluate reconstruction error on fault data
fault_loader = DataLoader(fault_dataset, batch_size=32, shuffle=False)
model.eval()
reconstruction_errors = []
with torch.no_grad():
    for data in fault_loader:
        output = model(data)
        loss = criterion(output, data)
        reconstruction_errors.append(loss.item())

# Print the average reconstruction error on fault data
print(f"Average reconstruction error on fault data: {np.mean(reconstruction_errors):.4f}")


Epoch [1/20], Loss: 0.1636
Epoch [2/20], Loss: 0.0438
Epoch [3/20], Loss: 0.0323
Epoch [4/20], Loss: 0.0253
Epoch [5/20], Loss: 0.0240
Epoch [6/20], Loss: 0.0232
Epoch [7/20], Loss: 0.0226
Epoch [8/20], Loss: 0.0219
Epoch [9/20], Loss: 0.0221
Epoch [10/20], Loss: 0.0212
Epoch [11/20], Loss: 0.0198
Epoch [12/20], Loss: 0.0139
Epoch [13/20], Loss: 0.0101
Epoch [14/20], Loss: 0.0093
Epoch [15/20], Loss: 0.0083
Epoch [16/20], Loss: 0.0079
Epoch [17/20], Loss: 0.0076
Epoch [18/20], Loss: 0.0078
Epoch [19/20], Loss: 0.0071
Epoch [20/20], Loss: 0.0067
Average reconstruction error on fault data: 0.0117


## LIVE STREAMING DATA

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler

# Define the dataset class
class VibrationDataset(Dataset):
    def __init__(self, csv_file):
        data = pd.read_csv(csv_file)
        self.scaler = MinMaxScaler()
        self.data = self.scaler.fit_transform(data[['x', 'y', 'z']].values)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32)

# Define the autoencoder model with structure similar to previous code
class Autoencoder(nn.Module):
    def __init__(self, input_dim=3):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Load datasets for non-fault and fault
non_fault_dataset = VibrationDataset('non_fault_data.csv')
fault_dataset = VibrationDataset('peak_drop_fault_data.csv')

# Create data loaders
non_fault_loader = DataLoader(non_fault_dataset, batch_size=32, shuffle=True)
fault_loader = DataLoader(fault_dataset, batch_size=32, shuffle=False)

# Initialize the model, loss function, and optimizer
input_dim = 3
model = Autoencoder(input_dim=input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the autoencoder only on non-fault data
num_epochs = 100
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for data in non_fault_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(non_fault_loader):.4f}")


# Evaluate reconstruction error on fault data
model.eval()
reconstruction_errors = []
with torch.no_grad():
    for data in fault_loader:
        output = model(data)
        loss = criterion(output, data)
        reconstruction_errors.append(loss.item())

# Print the average reconstruction error on fault data
print(f"Average reconstruction error on non_fault data: {total_loss / len(non_fault_loader):.4f}")

# Set a threshold for anomaly detection (optional)
threshold = total_loss / len(non_fault_loader) # np.mean(reconstruction_errors) + 2 * np.std(reconstruction_errors)

# Displaying anomalies based on threshold
print("Anomalies detected in fault data:")
for idx, error in enumerate(reconstruction_errors):
    if error > threshold:
        print(f"Sample {idx} - Reconstruction Error: {error:.4f} (Above threshold)")
    else:
        print(f"Sample {idx} - Reconstruction Error: {error:.4f} (Normal)")


Epoch [10/100], Loss: 0.0026
Epoch [20/100], Loss: 0.0021
Epoch [30/100], Loss: 0.0003
Epoch [40/100], Loss: 0.0002
Epoch [50/100], Loss: 0.0002
Epoch [60/100], Loss: 0.0002
Epoch [70/100], Loss: 0.0002
Epoch [80/100], Loss: 0.0002
Epoch [90/100], Loss: 0.0002
Epoch [100/100], Loss: 0.0002
Average reconstruction error on non_fault data: 0.0002
Anomalies detected in fault data:
Sample 0 - Reconstruction Error: 0.0001 (Normal)
Sample 1 - Reconstruction Error: 0.0009 (Above threshold)
Sample 2 - Reconstruction Error: 0.0005 (Above threshold)
Sample 3 - Reconstruction Error: 0.0000 (Normal)
Sample 4 - Reconstruction Error: 0.0007 (Above threshold)
Sample 5 - Reconstruction Error: 0.0007 (Above threshold)
Sample 6 - Reconstruction Error: 0.0000 (Normal)
Sample 7 - Reconstruction Error: 0.0008 (Above threshold)
Sample 8 - Reconstruction Error: 0.0008 (Above threshold)
Sample 9 - Reconstruction Error: 0.0000 (Normal)
Sample 10 - Reconstruction Error: 0.0003 (Above threshold)
Sample 11 - Recon

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.optimizers import Adam

# Parameters for the vibrational data sequence
window_size = 100  # Length of the sequence window
n_features = 1     # Single feature per timestep

# Generate synthetic vibrational data for demonstration (Replace with your actual data)
# Example: frequencies = np.sin(np.linspace(0, 50, time_steps)) + np.random.normal(0, 0.5, time_steps)
time_steps = 1000
frequencies = np.random.rand(time_steps)  # Placeholder for actual frequency data

# Create sequences (X) and labels (y)
X = []
y = []

for i in range(0, len(frequencies) - window_size):
    X.append(frequencies[i:i + window_size])
    y.append(frequencies[i + window_size])  # Next value as label (adjust as needed)

X = np.array(X).reshape(-1, window_size, n_features)
y = np.array(y)

# Transformer Encoder Block
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = Add()([x, inputs])  # Residual connection

    # Feed Forward Network
    x = LayerNormalization(epsilon=1e-6)(x)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return Add()([x, inputs])  # Residual connection

# Model Parameters
head_size = 64
num_heads = 4
ff_dim = 64
dropout = 0.1

# Input Layer
inputs = Input(shape=(window_size, n_features))
x = inputs

# Stacking multiple transformer encoder blocks
for _ in range(3):  # Stack 3 encoder blocks
    x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

# Output Layer
outputs = Dense(1)(x)

# Create and compile the model
model = Model(inputs, outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Display model summary
model.summary()

# Fit the model on training data
history = model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)

# Visualization of the frequency data
plt.plot(frequencies)
plt.title('Frequency vs. Time')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()


In [1]:
# server_mse_stream.py (run this on Laptop A)

import numpy as np
import torch
import torch.nn as nn
from collections import deque
import time
from flask import Flask, jsonify
import threading

# Initialize Flask app
app = Flask(__name__)
latest_mse = None  # Global variable to store the latest MSE value
latest_prediction = None  # Global variable to store the latest prediction
latest_target = None  # Global variable to store the latest actual target

# Define the LSTM model for live anomaly detection
class LSTMModel(nn.Module):
    def __init__(self, input_size=3, hidden_size=50, output_size=3 * 20):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

# Initialize model, criterion, and optimizer
input_length = 80
output_length = 20
model = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
threshold = 0.25
streaming_data_buffer = deque(maxlen=input_length + output_length)

# Generate synthetic vibrational data
def get_live_vibration_data(time_steps=100, noise_level=0.1, fault=False):
    t = np.linspace(0, 10, time_steps)
    frequency = 0.5
    if fault:
        stretch_factor = 3 * np.linspace(0, 1, time_steps)
        x = np.sin(2 * np.pi * frequency * t * stretch_factor) + noise_level * np.random.randn(time_steps)
        y = np.sin(2 * np.pi * frequency * t * stretch_factor + np.pi / 4) + noise_level * np.random.randn(time_steps)
        z = np.sin(2 * np.pi * frequency * t * stretch_factor + np.pi / 2) + noise_level * np.random.randn(time_steps)
    else:
        x = np.sin(2 * np.pi * frequency * t) + noise_level * np.random.randn(time_steps)
        y = np.sin(2 * np.pi * frequency * t + np.pi / 4) + noise_level * np.random.randn(time_steps)
        z = np.sin(2 * np.pi * frequency * t + np.pi / 2) + noise_level * np.random.randn(time_steps)
    return np.stack([x, y, z], axis=-1)

# Flask route to retrieve the latest MSE, prediction, and target
@app.route('/get_mse', methods=['GET'])
def get_mse():
    return jsonify({
        'mse': latest_mse,
        'prediction': latest_prediction.tolist() if latest_prediction is not None else None,
        'target': latest_target.tolist() if latest_target is not None else None
    })

# Background thread to run live anomaly detection
def run_anomaly_detection():
    global latest_mse, latest_prediction, latest_target
    while True:
        new_data = get_live_vibration_data(fault=np.random.rand() < 0.1)  # 10% chance of fault
        streaming_data_buffer.extend(new_data)
        if len(streaming_data_buffer) >= input_length + output_length:
            current_window = torch.tensor([list(streaming_data_buffer)[:input_length]], dtype=torch.float32)
            with torch.no_grad():
                prediction = model(current_window)
                target = torch.tensor([list(streaming_data_buffer)[input_length:]], dtype=torch.float32).reshape(1, -1)
                mse = criterion(prediction, target).item()

                # Store the latest prediction and target
                latest_prediction = prediction.numpy()
                latest_target = target.numpy()

            latest_mse = mse
            print(f"Latest MSE: {latest_mse}")  # Print for server logs
            for _ in range(output_length):
                streaming_data_buffer.popleft()
        time.sleep(1)

# Start the background anomaly detection thread
anomaly_thread = threading.Thread(target=run_anomaly_detection)
anomaly_thread.start()

# Run Flask server
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)


 * Serving Flask app '__main__'
 * Debug mode: off


/var/folders/2w/cxhzvjr95x3gvgqw7pmmsklr0000gn/T/ipykernel_6359/1238952718.py:71: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  current_window = torch.tensor([list(streaming_data_buffer)[:input_length]], dtype=torch.float32)
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://10.0.8.178:5001
Press CTRL+C to quit


Latest MSE: 0.5363554358482361
Latest MSE: 0.5377585291862488
Latest MSE: 0.5163723230361938
Latest MSE: 0.5384135842323303


10.0.20.121 - - [27/Oct/2024 01:32:32] "GET /get_mse HTTP/1.1" 200 -


Latest MSE: 0.4948725402355194


...

10.0.20.121 - - [27/Oct/2024 01:33:54] "GET /get_mse HTTP/1.1" 200 -
10.0.20.121 - - [27/Oct/2024 01:33:54] "GET /get_mse HTTP/1.1" 200 -
10.0.20.121 - - [27/Oct/2024 01:33:54] "GET /get_mse HTTP/1.1" 200 -
10.0.20.121 - - [27/Oct/2024 01:33:54] "GET /get_mse HTTP/1.1" 200 -
10.0.20.121 - - [27/Oct/2024 01:33:54] "GET /get_mse HTTP/1.1" 200 -
10.0.20.121 - - [27/Oct/2024 01:33:54] "GET /get_mse HTTP/1.1" 200 -


Latest MSE: 0.4845324754714966


10.0.20.121 - - [27/Oct/2024 01:33:55] "GET /get_mse HTTP/1.1" 200 -
10.0.20.121 - - [27/Oct/2024 01:33:55] "GET /get_mse HTTP/1.1" 200 -
10.0.20.121 - - [27/Oct/2024 01:33:55] "GET /get_mse HTTP/1.1" 200 -
10.0.20.121 - - [27/Oct/2024 01:33:55] "GET /get_mse HTTP/1.1" 200 -
10.0.20.121 - - [27/Oct/2024 01:33:55] "GET /get_mse HTTP/1.1" 200 -
10.0.20.121 - - [27/Oct/2024 01:33:55] "GET /get_mse HTTP/1.1" 200 -


Latest MSE: 0.5168081521987915


Latest MSE: 0.5327398180961609
Latest MSE: 0.5029152035713196
Latest MSE: 0.5030854344367981
Latest MSE: 0.5511385798454285
Latest MSE: 0.5241408944129944
Latest MSE: 0.5064858794212341
Latest MSE: 0.527363657951355
Latest MSE: 0.5422964096069336
Latest MSE: 0.5137836933135986
Latest MSE: 0.5243823528289795
Latest MSE: 0.5264840722084045
Latest MSE: 0.5193078517913818
Latest MSE: 0.5223009586334229
Latest MSE: 0.46571680903434753
Latest MSE: 0.5294600129127502
Latest MSE: 0.5133415460586548
Latest MSE: 0.498892605304718
Latest MSE: 0.5287692546844482
Latest MSE: 0.5000757575035095
Latest MSE: 0.530951738357544
Latest MSE: 0.514231264591217
Latest MSE: 0.515406608581543
Latest MSE: 0.5325338840484619
Latest MSE: 0.5365896821022034
Latest MSE: 0.5134990215301514
Latest MSE: 0.5367869734764099
Latest MSE: 0.5212730765342712
Latest MSE: 0.4974510967731476
Latest MSE: 0.5530520081520081
Latest MSE: 0.5328093767166138
Latest MSE: 0.528690755367279
Latest MSE: 0.5236771106719971
Latest MSE: 0

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load the synthetic vibrational data
df = pd.read_csv('fault_data.csv')

# Extract features and reshape to [samples, time_steps, features]
data = df.iloc[:, :-1].values.reshape(-1, 100, 3)  # Reshape to [samples, 100, 3]

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data.reshape(-1, 3)).reshape(-1, 100, 3)

# Prepare input-output pairs for the autoencoder
X_seq = data_scaled  # Use the same data for input
X_train, X_test = train_test_split(X_seq, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Define the enhanced Autoencoder model with convolutional layers
class EnhancedAutoencoder(nn.Module):
    def __init__(self):
        super(EnhancedAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(3, 16, kernel_size=3, stride=1, padding=1),  # Input shape: [batch, 3, 100]
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # Downsample
            nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Flatten()  # Flatten for the dense layer
        )
        self.decoder = nn.Sequential(
            nn.Linear(32 * 25, 128),
            nn.ReLU(),
            nn.Linear(128, 32 * 50),
            nn.ReLU(),
            nn.Unflatten(1, (32, 50)),  # Reshape back
            nn.ConvTranspose1d(32, 16, kernel_size=2, stride=2),  # Upsample
            nn.ReLU(),
            nn.ConvTranspose1d(16, 3, kernel_size=2, stride=2),  # Final output
            nn.Sigmoid()  # Use sigmoid to match the normalized range
        )

    def forward(self, x):
        encoded = self.encoder(x.permute(0, 2, 1))  # Change shape to [batch, features, time]
        decoded = self.decoder(encoded)
        return decoded.permute(0, 2, 1)  # Change back to [batch, time, features]

# Instantiate the model, define loss function and optimizer
model = EnhancedAutoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the Autoencoder
num_epochs = 100
batch_size = 32  # Increased batch size

train_losses = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for i in range(0, len(X_train_tensor), batch_size):
        batch = X_train_tensor[i:i + batch_size]
        optimizer.zero_grad()
        outputs = model(batch)
        loss = criterion(outputs, batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / (len(X_train_tensor) // batch_size)
    train_losses.append(avg_loss)

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')

# Plot training losses
plt.plot(train_losses)
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# Evaluate the Autoencoder
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, X_test_tensor).item()

print(f'Test Reconstruction Loss: {test_loss:.4f}')

# Detect anomalies in the test set
reconstruction_losses = criterion(test_outputs, X_test_tensor).numpy()
threshold = np.percentile(reconstruction_losses, 95)  # Set threshold for anomaly detection

# Identify anomalies based on the threshold
anomalies = reconstruction_losses > threshold

print(f'Number of anomalies detected: {np.sum(anomalies)}')
